In [2]:
import os
input_dir="C:\\Users\\선주환\\Desktop\\dacon_segmentation\\open\\train_img"
target_dir="C:\\Users\\선주환\\Desktop\\dacon_segmentation\\open\\train_mask"  #annotaiton 한결과
img_size=(1024,1024)
num_classes =3 #클래싀 개수 애완동물이 속한 ,인접, 그외의것 총 3개
batch_size=32

input_img_paths=sorted([os.path.join(input_dir,fname) for fname  in os.listdir(input_dir)
                        if fname.endswith('.png')])   #jpg에 해당하는것만 가져와서 정렬한뒤에 input_img_paths에 저장한다

target_img_paths=sorted([os.path.join(target_dir,fname) for fname  in os.listdir(target_dir)
                         if fname.endswith('.png') and not fname.startswith('.')])   #jpg에 해당하는것만 가져와서 정렬한뒤에 input_img_paths에 저장한다


In [3]:
from IPython.display import Image, display
from tensorflow.keras.preprocessing.image import load_img
import PIL
from PIL import ImageOps

# display(Image(filename=input_img_paths[20]))
img=PIL.ImageOps.autocontrast(load_img(target_img_paths[20]))
# display(img)

#경계면이 흰색으로 칠해져있고 그외에는 회색 고양이는 검정색으로 되어있다.

In [4]:
import numpy as np
from tensorflow import keras

class OxfordPets(keras.utils.Sequence):
    def __init__(self,batch_size,img_size,input_img_paths,target_img_paths):
        self.batch_size=batch_size
        self.img_size=img_size
        self.input_img_paths=input_img_paths
        self.target_img_paths=target_img_paths
    def __len__(self):
        return len(self.target_img_paths)//self.batch_size   #batch_size란 parameteer를 업데이트 할때 사용할 데이터 개수를 의미한다. 
    def __getitem__(self, index):
        i=index*self.batch_size  #다음 index의 위치하는곳에다가 넘어가기위해서
        batch_input_img_paths=self.input_img_paths[i:i+self.batch_size]   #batch단위로 가져오니까 즉 32개단위로 가져와서 처리하기 때문에 batch단위로 img_path를 정의 i가 현재는  0~32까지 가져오고 그다음에는
        #넘어간다
        batch_target_img_paths=self.target_img_paths[i:i+self.batch_size]
        x=np.zeros((self.batch_size,)+self.img_size+(3,),dtype='float32')  #이곳에다가 이미지를 넣을것이다. batch단위로 x에다가 input을 다 넣을것이다. 
        for j,path in enumerate(batch_input_img_paths):
            img=load_img(path,targte_size=self.img_size)
            x[j]=img  #j는 해당하는 index이다. 그 index에 img를 넣어준다.
        y=np.zeros((self.batch_size,)+self.img_size+(1,),dtype='unit8')# 이미지가 3컬러니까 3이고 밑에서는 흑백이다.
        for j,path in enumerate(batch_target_img_paths):
            img=load_img(path,targte_size=self.img_size,color_mode='grayscale')  #아 이이미지가 컬러가 아니라 흑백이니까 1이야
            y[j]=np.expand_dims(img,2)  
            y[j]-=1  #123으로 label 되어있던걸 012 로 동작할수있게끔 해준다.           
        return x,y   #입력 x와 결과 y이다.

In [5]:
import random
val_samples=1000
random.Random(1337).shuffle(input_img_paths)
random.Random(1337).shuffle(target_img_paths)
train_input_img_paths=input_img_paths[:-val_samples]
train_target_img_paths=target_img_paths[:-val_samples]

val_input_img_paths=input_img_paths[-val_samples:]
val_target_img_paths=input_img_paths[-val_samples:]

train_gen=OxfordPets(batch_size,img_size,train_input_img_paths,train_target_img_paths)
val_gen=OxfordPets(batch_size,img_size,val_input_img_paths,val_target_img_paths)




In [6]:
####u-net 기반의 모델을 구성한다.
from tensorflow.keras import layers
def get_model(img_size,num_classes):
    inputs=keras.Input(shape=img_size+(3,))  #+(3,) 인 이유는 컬러이기 때문이다.
    #처음에는 입력은 input으로 받아와서 U구조로 down 으로 내려간다.
    x=layers.Conv2D(32,3,strides=2,padding='same')(inputs)
    x=layers.BatchNormalization()(x)
    x=layers.Activation('relu')(x)
    previous_block_activation =x
    #down sampling 작업이다.
    for filters in [64,128,256]:
        x=layers.Activation('relu')(x)
        x=layers.SeparableConv2D(filters,3,padding='same')(x)
        x=layers.BatchNormalization()(x)
        
        x=layers.Activation('relu')(x)
        x=layers.SeparableConv2D(filters,3,padding='same')(x)
        x=layers.BatchNormalization()(x)

        x=layers.MaxPooling2D(3,strides=2,padding='same')(x)
        residual=layers.Conv2D(filters,1,strides=2,padding='same')(previous_block_activation)
        x=layers.add([x,residual])
        previous_block_activation=x  #다음 레지듀얼때 쓰기위해서x값을 저장해둔다.
    #up sampling작업
    for filters in[256,128,64,32]:
        x=layers.Activation('relu')(x)
        x=layers.Conv2DTranspose(filters,3,padding='same')(x)   #down은 separableconv2d 인데 up은 다르다
        x=layers.BatchNormalization()(x)
        
        x=layers.Activation('relu')(x)
        x=layers.Conv2DTranspose(filters,3,padding='same')(x)
        x=layers.BatchNormalization()(x)

        #48분까지 들음
        x=layers.UpSampling2D(2)(x)  #upsampling에 해당하는 부분이다.
        
        
        residual=layers.UpSampling2D(2)(previous_block_activation)
        residual=layers.Conv2D(filters,1,padding='same')(residual)
        x=layers.add([x,residual])
        previous_block_activation=x  #다음 레지듀얼때 쓰기위해서x값을 저장해둔다.
    outputs=layers.Conv2D(num_classes,3,activation='softmax',padding='same')(x)
    model=keras.Model(inputs,outputs)
    return model

model=get_model(img_size,num_classes)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 1024, 1024,  0           []                               
                                 3)]                                                              
                                                                                                  
 conv2d (Conv2D)                (None, 512, 512, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 512, 512, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [9]:
model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy')
callbacks=[keras.callbacks.ModelCheckPoint('oxford_segmetation.h5',save_best_only=True)]
epochs=10
model.fit(train_gen,epochs=epochs,validation_data=val_gen,callbacks=callbacks)


AttributeError: module 'keras.api._v2.keras.callbacks' has no attribute 'ModelCheckPoint'